<a href="https://colab.research.google.com/github/Seb85vickz/CMP7005-Programming-for-Data-Analysis/blob/main/streamlit_app_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Streamlit app: Advanced Multi-CSV Explorer

This notebook provides the Streamlit app file and instructions to run it in a local environment or inside Colab (with limitations).

Files created:

- `/mnt/data/streamlit_app.py` : the Streamlit application

You can download the app file directly from the links at the end of this notebook.

This cell will create the Streamlit application file at `/mnt/data/streamlit_app.py` with the provided Python code. This file contains the logic for the multi-CSV explorer.

In [8]:
%%writefile /mnt/data/streamlit_app.py

import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from io import StringIO
import base64
from collections import defaultdict
import re
import numpy as np
from sklearn.preprocessing import MinMaxScaler

st.set_page_config(layout='wide', page_title='Advanced Multi-CSV Explorer')

def get_table_download_link(df, filename='data.csv', text='Download data as CSV'):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()  # some strings <-> bytes conversions necessary here
    href = f'<a href="data:file/csv;base64,{b64}" download="{filename}">{text}</a>'
    return href

def df_info(df):
    info = pd.DataFrame({
        'Column': df.columns,
        'Non-Null Count': df.count().values,
        'Data Type': df.dtypes.values
    })
    return info

def clean_column_name(col_name):
    # Remove special characters and replace spaces with underscores
    cleaned_name = re.sub(r'[^a-zA-Z0-9_]', '', col_name.replace(' ', '_'))
    return cleaned_name

@st.cache_data
def load_data(uploaded_file):
    # Try decoding with utf-8 first, then fall back to latin-1
    try:
        return pd.read_csv(uploaded_file)
    except UnicodeDecodeError:
        uploaded_file.seek(0)  # Reset file pointer
        return pd.read_csv(uploaded_file, encoding='latin-1')

# State management for file uploads
if 'dfs' not in st.session_state:
    st.session_state.dfs = {}
if 'file_names' not in st.session_state:
    st.session_state.file_names = {}

# --- Sidebar for file uploads --- #
st.sidebar.header('Upload CSV Files')

uploaded_files = st.sidebar.file_uploader("Choose CSV files", type="csv", accept_multiple_files=True)

if uploaded_files:
    for uploaded_file in uploaded_files:
        file_id = uploaded_file.name # Use filename as a unique ID
        if file_id not in st.session_state.dfs:
            df = load_data(uploaded_file)
            if df is not None:
                st.session_state.dfs[file_id] = df
                st.session_state.file_names[file_id] = uploaded_file.name
            else:
                st.sidebar.error(f"Could not load {uploaded_file.name}")

# Remove files button
if st.session_state.dfs:
    st.sidebar.subheader("Remove Loaded Files")
    files_to_remove = st.sidebar.multiselect("Select files to remove", options=list(st.session_state.file_names.values()))
    if st.sidebar.button("Remove Selected Files"):
        for fn in files_to_remove:
            file_id_to_remove = next((id for id, name in st.session_state.file_names.items() if name == fn), None)
            if file_id_to_remove and file_id_to_remove in st.session_state.dfs:
                del st.session_state.dfs[file_id_to_remove]
                del st.session_state.file_names[file_id_to_remove]
        st.rerun()

# --- Main content --- #
st.title('Advanced Multi-CSV Explorer')

if not st.session_state.dfs:
    st.info("Please upload one or more CSV files from the sidebar to begin.")
else:
    tab_names = list(st.session_state.file_names.values()) + ['Combined Analysis']
    tabs = st.tabs(tab_names)

    # Display individual dataframes and their summaries
    for i, file_id in enumerate(st.session_state.dfs.keys()):
        with tabs[i]:
            st.header(f'File: {st.session_state.file_names[file_id]}')
            df = st.session_state.dfs[file_id]

            # Data Overview
            st.subheader('Data Overview')
            st.write(f'Shape: {df.shape[0]} rows, {df.shape[1]} columns')
            st.write('First 5 Rows:')
            st.dataframe(df.head())
            st.write('Column Information:')
            st.dataframe(df_info(df))

            # Missing Values
            st.subheader('Missing Values')
            missing_values = df.isnull().sum()
            missing_percentage = (df.isnull().sum() / len(df)) * 100
            missing_df = pd.DataFrame({
                'Missing Count': missing_values,
                'Missing Percentage': missing_percentage
            }).sort_values(by='Missing Count', ascending=False)
            st.dataframe(missing_df[missing_df['Missing Count'] > 0])

            if not df.empty and df.select_dtypes(include=np.number).empty:
                st.warning("This DataFrame contains no numeric columns for descriptive statistics.")
            elif not df.empty:
                st.subheader('Descriptive Statistics (Numeric Columns)')
                st.dataframe(df.describe())

            st.markdown(get_table_download_link(df, filename=f'{st.session_state.file_names[file_id].replace(".csv", "_processed.csv")}', text='Download this DataFrame'), unsafe_allow_html=True)

    # Combined Analysis Tab
    with tabs[-1]:
        st.header('Combined Data Analysis')

        if len(st.session_state.dfs) < 2:
            st.info("Upload at least two CSV files to perform combined analysis.")
        else:
            # Dropdown to select columns for merging
            st.subheader('Merge DataFrames')
            selected_merge_cols = st.multiselect(
                "Select common columns to merge on (optional):",
                options=list(set.intersection(*[set(df.columns) for df in st.session_state.dfs.values()]))
            )

            combined_df = pd.DataFrame()
            if not st.session_state.dfs:
                st.error("No dataframes available for combination.")
            else:
                dfs_to_combine = list(st.session_state.dfs.values())
                file_names_list = list(st.session_state.file_names.values())

                if selected_merge_cols:
                    # Initial merge
                    combined_df = dfs_to_combine[0]
                    for k in range(1, len(dfs_to_combine)):
                        combined_df = pd.merge(combined_df, dfs_to_combine[k], on=selected_merge_cols, how='outer', suffixes=(f'_{file_names_list[k-1].replace(".csv", "")}', f'_{file_names_list[k].replace(".csv", "")}{k}'))
                else:
                    # Concatenate if no merge columns selected or merge not possible
                    try:
                        combined_df = pd.concat(dfs_to_combine, ignore_index=True)
                    except Exception as e:
                        st.warning(f"Could not concatenate DataFrames directly due to differing columns. Using first DataFrame for initial combined analysis. Error: {e}")
                        combined_df = dfs_to_combine[0] # Fallback to first DF if concat fails

            if not combined_df.empty:
                st.write("Combined DataFrame Head:")
                st.dataframe(combined_df.head())

                st.subheader('Correlation Heatmap (Combined Numeric Data)')
                numeric_cols_combined = combined_df.select_dtypes(include=np.number).columns
                if len(numeric_cols_combined) > 1:
                    corr_matrix = combined_df[numeric_cols_combined].corr()
                    fig_corr = px.imshow(corr_matrix, text_auto=True, color_continuous_scale='Viridis')
                    st.plotly_chart(fig_corr, use_container_width=True)
                else:
                    st.info("Not enough numeric columns in the combined data for a correlation heatmap.")

                st.subheader('Distribution of Numeric Columns (Combined Data)')
                numeric_cols = combined_df.select_dtypes(include=np.number).columns.tolist()
                if numeric_cols:
                    selected_numeric_col_dist = st.selectbox('Select a numeric column for distribution:', numeric_cols, key='combined_dist_col')
                    if selected_numeric_col_dist:
                        fig_dist = px.histogram(combined_df, x=selected_numeric_col_dist, marginal="box", nbins=30, title=f'Distribution of {selected_numeric_col_dist}')
                        st.plotly_chart(fig_dist, use_container_width=True)
                else:
                    st.info("No numeric columns in the combined data to display distributions.")

                st.subheader('Categorical Column Value Counts (Combined Data)')
                categorical_cols = combined_df.select_dtypes(include='object').columns.tolist()
                if categorical_cols:
                    selected_cat_col_vc = st.selectbox('Select a categorical column for value counts:', categorical_cols, key='combined_cat_vc')
                    if selected_cat_col_vc:
                        value_counts = combined_df[selected_cat_col_vc].value_counts().reset_index()
                        value_counts.columns = [selected_cat_col_vc, 'Count']
                        fig_vc = px.bar(value_counts.head(20), x=selected_cat_col_vc, y='Count', title=f'Value Counts for {selected_cat_col_vc}')
                        st.plotly_chart(fig_vc, use_container_width=True)
                else:
                    st.info("No categorical columns in the combined data to display value counts.")

                # Pairplot for numerical columns
                if len(numeric_cols) > 1:
                    st.subheader('Scatter Matrix (Pairplot) of Numeric Columns')
                    pairplot_cols = st.multiselect('Select up to 5 numeric columns for pairplot:', numeric_cols, default=numeric_cols[:min(5, len(numeric_cols))], key='pairplot_cols')
                    if pairplot_cols:
                        try:
                            fig_pair = px.scatter_matrix(combined_df, dimensions=pairplot_cols)
                            st.plotly_chart(fig_pair, use_container_width=True)
                        except Exception as e:
                            st.error(f"Error generating pairplot: {e}")

                st.markdown(get_table_download_link(combined_df, filename='combined_data_processed.csv', text='Download Combined DataFrame'), unsafe_allow_html=True)

Writing /mnt/data/streamlit_app.py


In [9]:
# Show the created streamlit app path and preview the first 200 lines
from pathlib import Path
p = Path('/mnt/data/streamlit_app.py')
print('App exists:', p.exists())
print('-'*40)
print('\n'.join(open(p).read().splitlines()[:200]))

App exists: True
----------------------------------------

import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from io import StringIO
import base64
from collections import defaultdict
import re
import numpy as np
from sklearn.preprocessing import MinMaxScaler

st.set_page_config(layout='wide', page_title='Advanced Multi-CSV Explorer')

def get_table_download_link(df, filename='data.csv', text='Download data as CSV'):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()  # some strings <-> bytes conversions necessary here
    href = f'<a href="data:file/csv;base64,{b64}" download="{filename}">{text}</a>'
    return href

def df_info(df):
    info = pd.DataFrame({
        'Column': df.columns,
        'Non-Null Count': df.count().values,
        'Data Type': df.dtypes.values
    })
    return info

def clean_column_name(col_name):
    # Remove special characters and replace spaces with underscores
    cle

## How to run the Streamlit app locally

1. Ensure you have Python 3.8+ and pip installed.
2. Install required packages:

```
pip install streamlit pandas plotly scikit-learn
```

3. Run the app from the folder that contains your CSV files (or set the Data directory in the sidebar):

```
streamlit run /mnt/data/streamlit_app.py
```

If you're using Google Colab, running Streamlit requires extra steps (using `ngrok` or `localtunnel`) and is not covered here. It's recommended to run locally for full interactivity.


## Files created
Use the links below to download the app file and the notebook itself.

In [10]:
from IPython.display import FileLink, FileLinks
print('Streamlit app file:')
print(FileLink('/mnt/data/streamlit_app.py'))
print('\nNotebook file:')
print(FileLink('/mnt/data/streamlit_app_notebook.ipynb'))

Streamlit app file:
/mnt/data/streamlit_app.py

Notebook file:
/mnt/data/streamlit_app_notebook.ipynb
